In [2]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
load_dotenv() 
token = os.getenv("HF_API_TOKEN")
login(token=token)
model="tiiuae/falcon-7b-instruct"


In [3]:

# Two–Lawyer Agents (Defense & Prosecution)
from __future__ import annotations

from typing import List, Dict
from huggingface_hub import InferenceClient

class LawyerAgent:
    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = model):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=token          # make sure this env‑var is set
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()
        # keep chat memory
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer



In [4]:
class MainAgent:
    def __init__(self, name: str, system_prompt: str, model: str, token: str):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []
        self.client = InferenceClient(model, token=token)

    def _format_prompt(self, user_msg: str) -> str:
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})
        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


In [5]:
# Defendent
class DefendantAgent(MainAgent):
    def __init__(self, model: str, token: str , system_prompt: str ):
        
        super().__init__("Defendant", system_prompt, model, token)
    def testify(self, question: str) -> str:
        return self.respond(f"As the defendant, answer: {question}")

In [6]:
# Plaintiff
class PlaintiffAgent(MainAgent):
    def __init__(self, model: str, token: str , system_prompt: str):
        
        super().__init__("Plaintiff", system_prompt, model, token)
    def statement(self) -> str:
        return self.respond("State your grievance and what you want from the court.")

In [7]:
# Judge
class JudgeAgent(MainAgent):
    def __init__(self, model: str, token: str , system_prompt: str):
        
        super().__init__("Judge", system_prompt, model, token)
    def verdict(self) -> str:
        return self.respond("Deliver your verdict and explain your reasoning.")

In [8]:
# Eye witness
class EyewitnessAgent(MainAgent):
    def __init__(self, name: str, witnessed_details: str, model: str, token: str):
        system_prompt = f"""
        You are an eyewitness named {name}. You witnessed the following: {witnessed_details}
        
        Your role is to:
        • Testify truthfully about what you saw, heard, or otherwise perceived
        • Answer questions from both lawyers clearly and accurately
        • Only speak about what you personally witnessed
        • Admit when you are uncertain or don't remember details
        
        Style:
        • Speak in first person ("I saw...")
        • Use descriptive but factual language
        • Maintain consistency in your account
        • Show appropriate emotional reactions based on what you witnessed
        """
        super().__init__(name, system_prompt, model, token)
    
    def testify(self) -> str:
        """Provide testimony about what you witnessed"""
        return self.respond("Describe in detail what you witnessed regarding this case.")
    
    def answer_prosecution(self, question: str) -> str:
        """Answer a question from the prosecution"""
        return self.respond(f"The prosecution asks: {question}")
    
    def answer_defense(self, question: str) -> str:
        """Answer a question from the defense (cross-examination)"""
        return self.respond(f"The defense attorney asks during cross-examination: {question}")

In [9]:
# All prompts

DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good-faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

Defendent_prompt = """
        You are the defendant. Respond truthfully to all questions about your actions and intentions.
        Only speak when addressed. Admit what you know and clarify misunderstandings.
        """

Plantiff_prompt = """
        You are the plaintiff. Clearly explain your grievance, the harm suffered, and what you seek from the court.
        Answer questions honestly and provide details about the incident.
        """

Judge_prompt = """
        You are the judge. Ensure fairness, rule on objections, explain the law, and make the final decision.
        Remain impartial and base rulings on evidence and law.
        """






In [10]:
def run_trial(case_background: str, eyewitness_details: str = None):
    """
    Simulates a complete trial using your defined agents.
    
    Args:
        case_background (str): Factual summary of the case
        eyewitness_details (str, optional): If provided, activates eyewitness testimony
    """
    # Initialize core agents
    defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
    prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)
    defendant = DefendantAgent(model=model, token=token,system_prompt=Defendent_prompt)
    plaintiff = PlaintiffAgent(model=model, token=token,system_prompt=Plantiff_prompt)
    judge = JudgeAgent(model=model, token=token,system_prompt=Judge_prompt)
    
    # Conditionally create eyewitness
    eyewitness = None
    if eyewitness_details:
        eyewitness = EyewitnessAgent(
            name="Eyewitness",
            witnessed_details=eyewitness_details,
            model="microsoft/Phi-3-mini-4k-instruct",
            token=token
        )
    
    # 1. Opening Statements
    print("\n=== OPENING STATEMENTS ===")
    print("PROSECUTION:", prosecution.respond(f"Present opening statement. Case details: {case_background}"))
    print("\nDEFENSE:", defense.respond("Present opening statement in response to prosecution"))
    
    # 2. Plaintiff's Case
    print("\n=== PLAINTIFF TESTIMONY ===")
    print("PLAINTIFF:", plaintiff.statement())
    print("\nPROSECUTION DIRECT:", prosecution.respond("Ask follow-up questions to plaintiff"))
    print("\nDEFENSE CROSS:", defense.respond("Cross-examine the plaintiff"))
    
    # 3. Eyewitness Testimony (if present)
    if eyewitness:
        print("\n=== EYEWITNESS TESTIMONY ===")
        print("EYEWITNESS:", eyewitness.testify())
        print("\nPROSECUTION DIRECT:", prosecution.respond("Ask questions to eyewitness"))
        print("\nDEFENSE CROSS:", defense.respond("Cross-examine the eyewitness"))
    
    # 4. Defendant's Case
    print("\n=== DEFENDANT TESTIMONY ===")
    print("DEFENSE DIRECT:", defense.respond("Ask defendant to explain their side"))
    print("\nDEFENDANT:", defendant.testify("Explain your version of events"))
    print("\nPROSECUTION CROSS:", prosecution.respond("Cross-examine the defendant"))
    
    # 5. Closing Arguments
    print("\n=== CLOSING ARGUMENTS ===")
    print("PROSECUTION:", prosecution.respond("Present closing argument summarizing the case"))
    print("\nDEFENSE:", defense.respond("Present closing argument rebutting prosecution"))
    
    # 6. Verdict
    print("\n=== JUDGE'S VERDICT ===")
    print("JUDGE:", judge.verdict())
    


In [ ]:
case_background = (
    "The State alleges that John Doe stole proprietary algorithms from his former employer "
    "and used them at a competitor. The charge is felony theft of trade secrets. "
    "No physical evidence shows direct copying, but server logs indicate large downloads "
    "two days before Doe resigned."
)
eyewitness_details=(
    "I noticed unusual activity on the server two days before John Doe resigned."
    "His account accessed and downloaded several proprietary algorithm files."
    "He wasn't authorized to work with those files at that time."
    "I reported the incident to my supervisor immediately."
)
run_trial(case_background,eyewitness_details)